In [1]:
import tensorflow as tf
import numpy as np
import pickle
import json
import nltk
import random
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [2]:
NEXT_QNT = NEURONS = EPOCHS = 0
with open("../Data/iniV2.pickle", "rb") as f:
    NEXT_QNT , NEURONS, EPOCHS = pickle.load(f)
print(NEURONS,EPOCHS)

64 100


In [54]:
words = []
labels = []
docs_x = []
docs_y = []

with open("../Data/intents.json","r",encoding='utf-8') as file:
    data = json.load(file)

for i,intent in enumerate(data["intents"]):
    intent["tag"] = "tag"+str(i)

with open("../Data/intents.json","w",encoding='utf-8') as file:
    json.dump(data,file,indent=3,ensure_ascii=False)

In [55]:
stopwords = nltk.corpus.stopwords.words('portuguese')
len(stopwords)

204

In [56]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])

    if intent["tag"] not in labels:
        labels.append(intent["tag"])

ponctuations = ["?","'",'"',"!",".",","]

words = [w for w in words if w.lower() not in stopwords]
words = [stemmer.stem(w.lower()) for w in words if w not in ponctuations]
words = sorted(list(set(words)))
labels = sorted(labels)
print(len(words))

493


In [57]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)
N_CLASSES = len(output[0])
SHAPE = len(training[0])

In [58]:
def dense_layers(inputs):
    x = tf.keras.layers.Dense(NEURONS,activation='relu')(inputs)
    x = tf.keras.layers.Dense(NEURONS,activation='relu')(x)
    x = tf.keras.layers.Dense(NEURONS,activation='relu')(x)
    return x

def classfier_layer(x,N_CLASSES):
    x = tf.keras.layers.Dense(N_CLASSES,activation='softmax',name='classification')(x)
    return x

def final_model(inputs,N_CLASSES):
    dense = dense_layers(inputs)
    
    classfier = classfier_layer(dense,N_CLASSES)
    
    model = tf.keras.Model(inputs=inputs,outputs=classfier)
    
    return model
    
def define_and_compile_model(SHAPE,N_CLASSES):
    inputs = tf.keras.layers.Input(shape=(SHAPE,))
    
    # create the model
    model = final_model(inputs,N_CLASSES)
    
    # compile your model
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics = {'classification' : 'accuracy'})

    return model

In [59]:
model = define_and_compile_model(SHAPE,N_CLASSES)
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 493)]             0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                31616     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                4160      
_________________________________________________________________
classification (Dense)       (None, 173)               11245     
Total params: 47,021
Trainable params: 47,021
Non-trainable params: 0
_________________________________________________________________


In [62]:
model.fit(training,output,epochs=EPOCHS)

Epoch 1/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0320 - accuracy: 0.1495
Epoch 2/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0317 - accuracy: 0.1495
Epoch 3/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0313 - accuracy: 0.1655
Epoch 4/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0310 - accuracy: 0.1690
Epoch 5/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0306 - accuracy: 0.1797
Epoch 6/100
18/18 [==============================] - 0s 2ms/step - loss: 0.0301 - accuracy: 0.2135
Epoch 7/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0297 - accuracy: 0.2171
Epoch 8/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0292 - accuracy: 0.2295
Epoch 9/100
18/18 [==============================] - 0s 3ms/step - loss: 0.0287 - accuracy: 0.2509
Epoch 10/100
18/18 [==============================] - 0s 4ms/step - loss: 0.0281 - accuracy: 0.2544
Epoch 11/

In [79]:
def bag_of_words(s):
    bag = [0 for _ in range(len(words))]

    ponctuations = ["?","'",'"',"!",".",","]

    s_words = nltk.word_tokenize(s)
    s_words = [w for w in s_words if w.lower() not in stopwords]
    s_words = [stemmer.stem(word.lower()) for word in s_words if word not in ponctuations]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    bag = np.array(bag)
    return bag

bag = bag_of_words("como você tá?")
bag.shape

(493,)

In [80]:
bag = bag.reshape((1,SHAPE))
predictions = model.predict(bag)
results_index = np.argmax(predictions)

tag = labels[results_index]

for tg in data['intents']:
    if tg['tag'] == tag:
        responses = tg['responses']
        current_tag = tg

response = random.choice(responses)
confidence = predictions[0][results_index]
print(response,confidence)

Me sinto otima,e você como se sente? 0.66586596


In [81]:
model.save("model.h5")

In [83]:
with open("../Data/dataV2.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)    

In [ ]:
with open("Chatbot/Data/iniV2.pickle", "wb") as f:
    pickle.dump((next_qnt, self.NEURONS, self.EPOCHS), f)